In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'  
import sys
print(sys.version)

3.8.3 (default, Jul  2 2020, 16:21:59) 
[GCC 7.3.0]


In [10]:
##### tools

class qstate:
    def __init__(self, arr, uppN, lowN):
        # define an array for the coefficients of each combined basis state, 
        #   as well as number of bits for upper and lower register (to know where to cut)
        self.arr = arr
        self.uppN = uppN
        self.lowN = lowN
    def printC(self):
        # testing function that prints each basis state coefficient
        for i in range(0, len(self.arr)):
            print("Coefficient of {val}: {c}".format(val = i, c = self.arr[i]))

def norm(state):
    # this normalizes a given state
    sqsum = 0
    for c in state.arr:
        sqsum += c**2
    newArr = state.arr/(np.sqrt(sqsum))
    return qstate(newArr, state.uppN, state.lowN)

def measupp(state):
    # measures upper register of a combined state
    newArr = np.zeros(len(state.arr))
    r = np.random.random()
    x0 = None
    # utilizing random number check, subtractive rolling into consecutive indexes until one fits within probability
    for i in range(0, len(state.arr)):
        if state.arr[i]**2 > r:
            x0 = i // (2**(state.uppN))
            print("Upper register measured as: {c}".format(c = x0))
            break
        else:
            r -= state.arr[i]**2
    # "new" array is a superposition of only those original states whose upper register matches what was measured
    for i in range(0, len(state.arr)):
        if (i // (2**(state.uppN))) == x0:
            newArr[i] = state.arr[i]
    new = qstate(newArr, state.uppN, state.lowN)
    return norm(new)

def measlow(state):
    # measures lower register of a combined state
    newArr = np.zeros(len(state.arr))
    r = np.random.random()
    y0 = None
    # utilizing random number check, subtractive rolling into consecutive indexes until one fits within probability
    for i in range(0, len(state.arr)):
        if state.arr[i]**2 > r:
            y0 = i % (2**(state.uppN))
            print("Lower register measured as: {c}".format(c = y0))
            break
        else:
            r -= state.arr[i]**2
    # "new" array is a superposition of only those original states whose lower register matches what was measured
    for i in range(0, len(state.arr)):
        if (i % (2**(state.uppN))) == y0:
            newArr[i] = state.arr[i]
    new = qstate(newArr, state.uppN, state.lowN)
    return norm(new)
    
def mod2add(a, b, n):
    # bitwise modulo 2 addition
    sum = 0 
    for i in range(0, n):
        sum += ((a//(2**i) + b//(2**i))%2)*(2**i)
    return sum
    
def bip(a, b, n):
    # bitwise inner product (with modulo 2 addition)
    modsum = 0 
    for i in range(0, n):
        modsum += ( (a//(2**i))%2 )*( (b//(2**i))%2 )
    return modsum%2

def xat(state, bit):
    # enacts a single-bit x-gate (bit flip) on a specific bit (counting up)
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        above = i // (2**(bit+1))
        below = i % (2**bit)
        atbit = i // (2**(bit)) % (2**bit)
        newNum = above*(2**(bit+1)) + (1-atbit)*(2**bit) + below
        newArr[newNum] += state.arr[i]
    return qstate(newArr, state.uppN, state.lowN)

def zat(state, bit):
    # enacts a single-bit z-gate (phase flip) on a specific bit (counting up)
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        atbit = i // (2**(bit)) % (2**bit)
        newArr[i] = state.arr[i]*(-1**atbit)
    return qstate(newArr, state.uppN, state.lowN)

def hat(state, bit):
    # enacts a single-bit hadamard on a specific bit (counting up)
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        above = i // (2**(bit+1))
        below = i % (2**bit)
        atbit = i // (2**(bit)) % (2**bit)
        new0 = above*(2**(bit+1)) + below
        new1 = above*(2**(bit+1)) + 2**(bit) + below
        newArr[new0] += state.arr[i]/np.sqrt(2)
        newArr[new1] += state.arr[i]*((-1)**atbit)/np.sqrt(2)
    return qstate(newArr, state.uppN, state.lowN)

def cnot(state, cbit, tbit):
    # enacts a c-not gate with specified control and target bits
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        above = i // (2**(tbit+1))
        below = i % (2**tbit)
        attbit = i // (2**(tbit)) % (2**tbit)
        atcbit = i // (2**(cbit)) % (2**cbit)
        newNum = 0
        if(atcbit == 1):
            newNum = above*(2**(tbit+1)) + (1-attbit)*(2**tbit) + below
        else:
            newNum = above*(2**(tbit+1)) + attbit*(2**tbit) + below
        newArr[newNum] += state.arr[i]
    return qstate(newArr, state.uppN, state.lowN)

def hupp(state):
    # enacts a bitwise hadamard gate on the upper register
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        for y in range(0, 2**state.uppN):
            newNum = int(y*(2**state.lowN) + lowNum)
            newArr[newNum] += (state.arr[i]*((-1)**(bip(uppNum, y, state.uppN))))/ (2**(state.uppN/2))
    return qstate(newArr, state.uppN, state.lowN)
     
def hlow(state):
    # enacts a bitwise hadamard gate on the lower register
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        for y in range(0, 2**state.lowN):
            newNum = int(uppNum*(2**state.lowN) + y)
            newArr[newNum] += (state.arr[i]*((-1)**(bip(y, lowNum, state.lowN))))/ (2**(state.lowN/2))
    return qstate(newArr, state.uppN, state.lowN)
     
def U(state, f):
    # quantum function black box acting upon lower register such that |y⟩ -> |y mod2+ f(x)⟩, for a given f.
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        fval = f(uppNum)
        newNum = int(uppNum*(2**state.lowN) + mod2add(lowNum, fval, state.lowN))
        newArr[newNum] += state.arr[i]
    return qstate(newArr, state.uppN, state.lowN)
    
def UShor(state, n, a):
    # modified black box specifically to take shor algorithim paramters, work with fShor
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        fval = fShor(uppNum, n, a)
        newNum = int(uppNum*(2**state.lowN) + mod2add(lowNum, fval, state.lowN))
        newArr[newNum] += state.arr[i]
    return qstate(newArr, state.uppN, state.lowN)

### some example functions for Deutsch and Simon's algorithms

def fDeutsch1(x):
    return 0

def fDeutsch2(x):
    if(x == 0):
        return 0
    else:
        return 1

def fDeutsch3(x):
    if(x == 0):
        return 1
    else:
        return 0

def fDeutsch4(x):
    return 1

def fSimon3(x):
    # just an example of a function that may be used in Simon's Algorithm
    if(x == 0 or x == 3):
        return 3
    elif(x == 1 or x == 2):
        return 2
    elif(x == 4 or x == 7):
        return 0
    elif(x == 5 or x == 6):
        return 1
    else:
        print("Error in black box function - input must be 3 bits")
        return None
###

def fShor(x, n, a):
    # quantum function for Shor's algorithm 
    w = a
    fx = 1
    for i in range (0, n):
        if((x//(2**i))%2):
            fx *= w
        w = w**2
    return w

def QFTupp(state):
    # Quantum fourier transform on upper register (creates issues due to complex numbers, don't think i will implement)
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        for y in range(0, 2**state.uppN):
            newNum = int(y*(2**state.lowN) + lowNum)
            newArr[newNum] += state.arr[i]*np.math.exp(2*np.pi*1j*uppNum*y / (2**state.uppN))
        return qstate(newArr, state.uppN, state.lowN)

def QFTlow(state):
    # Quantum fourier transform on lower register (creates issues due to complex numbers, don't think i will implement)
    newArr = np.zeros(len(state.arr))
    for i in range(0, len(state.arr)):
        uppNum = i // (2**(state.lowN))
        lowNum = i % (2**(state.lowN))
        for y in range(0, 2**state.lowN):
            newNum = int(uppNum*(2**state.lowN) + y)
            newArr[newNum] += state.arr[i]*np.math.exp(2*np.pi*1j*lowNum*y / (2**state.lowN))
        return qstate(newArr, state.uppN, state.lowN)
      
##### algorithms
        
def Deutsch(func):
    print("Starting up Deutsch Algorithm.")
    print("--------------------------------")
    runstate = qstate([0, 1, 0, 0], 1, 1)
    runstate = hupp(runstate)
    runstate = hlow(runstate)
    runstate = U(runstate, func)
    runstate = hupp(runstate)
    runstate = hlow(runstate)
    runstate = measupp(runstate)
    print("--------------------------------")
    print("Deutsch Algorithm: Run completed")
    
def Simon(n, func):
    print("Starting up Simon's Algorithm for {n} upper bits.".format(n = n))
    print("------------------------------------------")
    arr = np.zeros(2**(2*n-1))
    arr[0] = 1
    runstate = qstate(arr, n, n-1)
    runstate = hupp(runstate)
    runstate = U(runstate, func)
    runstate = measlow(runstate)
    print("------------------------------------------")
    runstate = hupp(runstate)
    runstate = measupp(runstate)
    print("------------------------------------------")
    print("Simon's Algorithm: Run completed")

######### this does not work due to issues processing complex numbers
def Shor(n, a):
    print("Starting up Shor's Algorithm for {n} upper bits, a = {a}.".format(n = n, a = a))
    print("-----------------------------------------")
    arr = np.zeros(2**(3*n//2))
    arr[0] = 1
    runstate = qstate(arr, n, n/2)
    runstate = UShor(runstate, n, a)
    runstate = measlow(runstate)
    print("------------------------------------------")
    runstate = QFTupp(runstate)
    runstate = measupp(runstate)
    print("------------------------------------------")
    print("Shor's Algorithm: Run completed")
    
    
##### testing